In [1]:
import tweepy
import json
import numpy as np
import pandas as pd

In [2]:
print('TweePy version is:', tweepy.__version__)

'4.10.0'

In [24]:
def authenticate(creds_path):
    '''
    read the credentials and authenticate to Twitter 
    :param creds_path: str, containing path to credentials file
    :return: api, the authenticated API object 
    '''
    with open(creds_path, mode='r') as f: creds = json.load(f)
    C_KEY, C_SECRET = creds['api_key'], creds['api_key_secret']
    A_TOKEN, A_TOKEN_SECRET = creds['access_token'], creds['access_token_secret']
    auth = tweepy.OAuthHandler(consumer_key=C_KEY, consumer_secret=C_SECRET)
    auth.set_access_token(key=A_TOKEN, secret=A_TOKEN_SECRET)
    api = tweepy.API(auth=auth, wait_on_rate_limit=True)
    return api

In [20]:
USR_NAME = '***'

api = authenticate(creds_path='path/to/creds.json')
user = api.get_user(screen_name=USR_NAME)

In [21]:
user.timeline()

[Status(_api=<tweepy.api.API object at 0x7f839b967250>, _json={'created_at': 'Sun Jun 05 19:17:22 +0000 2022', 'id': 1533528425234944000, 'id_str': '1533528425234944000', 'text': 'RT @Hardesty3Tina: https://t.co/UGiKemjSaI', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'Hardesty3Tina', 'name': 'Tina Hardesty', 'id': 944431067422969856, 'id_str': '944431067422969856', 'indices': [3, 17]}], 'urls': [], 'media': [{'id': 1533109594221854720, 'id_str': '1533109594221854720', 'indices': [19, 42], 'media_url': 'http://pbs.twimg.com/media/FUazDZ1WQAAqUnx.jpg', 'media_url_https': 'https://pbs.twimg.com/media/FUazDZ1WQAAqUnx.jpg', 'url': 'https://t.co/UGiKemjSaI', 'display_url': 'pic.twitter.com/UGiKemjSaI', 'expanded_url': 'https://twitter.com/Hardesty3Tina/status/1533109598432792576/photo/1', 'type': 'photo', 'sizes': {'thumb': {'w': 150, 'h': 150, 'resize': 'crop'}, 'large': {'w': 766, 'h': 960, 'resize': 'fit'}, 'medium': {'w': 766, 'h': 9